In [21]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from batcher import Container
import cifar10
%matplotlib inline

In [22]:
ims, labels = cifar10.get_train_data()
images = Container(ims, labels)

In [23]:
test_x, test_y = cifar10.get_test_data()

In [24]:
sess = tf.InteractiveSession()

In [25]:
channels = 3

In [26]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [27]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [28]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [29]:
conv1_filters = 96

W_conv1 = weight_variable([5, 5, channels, conv1_filters])
b_conv1 = bias_variable([conv1_filters])

h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_3x3(h_conv1)

In [30]:
conv2_filters = 128

W_conv2 = weight_variable([5, 5, conv1_filters, conv2_filters])
b_conv2 = bias_variable([conv2_filters])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_3x3(h_conv2)

In [31]:
conv3_filters = 256

W_conv3 = weight_variable([5, 5, conv2_filters, conv3_filters])
b_conv3 = bias_variable([conv3_filters])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_3x3(h_conv3)

In [32]:
fc_units = 2048

In [33]:
W_fc1 = weight_variable([4 * 4 * conv3_filters, fc_units])
b_fc1 = bias_variable([fc_units])

h_pool3_flat = tf.reshape(h_pool3, [-1, 4 * 4 * conv3_filters])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

In [34]:
W_fc2 = weight_variable([fc_units, fc_units])
b_fc2 = bias_variable([fc_units])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

In [37]:
W_logit = weight_variable([fc_units, 10])
b_logit = bias_variable([10])

y_conv = tf.matmul(h_fc2, W_logit) + b_logit

In [40]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(15000):
    batch_x, batch_y = images.next_batch(500)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_: batch_y})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch_x, y_: batch_y})

step 0, training accuracy 0.098
step 100, training accuracy 0.326
step 200, training accuracy 0.392
step 300, training accuracy 0.396
step 400, training accuracy 0.392
step 500, training accuracy 0.456
step 600, training accuracy 0.464
step 700, training accuracy 0.52
step 800, training accuracy 0.478
step 900, training accuracy 0.52
step 1000, training accuracy 0.502
step 1100, training accuracy 0.514
step 1200, training accuracy 0.504
step 1300, training accuracy 0.524
step 1400, training accuracy 0.558
step 1500, training accuracy 0.538
step 1600, training accuracy 0.55
step 1700, training accuracy 0.602
step 1800, training accuracy 0.576
step 1900, training accuracy 0.596
step 2000, training accuracy 0.56
step 2100, training accuracy 0.564
step 2200, training accuracy 0.618
step 2300, training accuracy 0.646
step 2400, training accuracy 0.622
step 2500, training accuracy 0.638
step 2600, training accuracy 0.632
step 2700, training accuracy 0.664
step 2800, training accuracy 0.618
s

In [41]:
chunk_size = 500

chunk_accuracies = [
    accuracy.eval(
        feed_dict={x: test_x[start:start+chunk_size], y_: test_y[start:start+chunk_size]}
    )
    for start in range(0,len(test_y),chunk_size)
]

acc = np.mean(chunk_accuracies)

print("test accuracy %g"%acc)

test accuracy 0.6178
